Write code to classify two famous celebrities:
We need to prepare the data to do so and also prepare the model as well.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [28]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [35]:
# ImageGenerators:
training_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
testing_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)

In [ ]:
training_image_data = training_image_generator.flow_from_directory('celeb/data/training',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (128,128))

testing_image_data = testing_image_generator.flow_from_directory('celeb/data/validation',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (128,128))

Found 31 images belonging to 2 classes.


Found 22 images belonging to 2 classes.


In [37]:
# Model creation:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape , activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())

# FC:
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))

model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,108,513 (4.23 MB)

 Trainable params: 1,108,513 (4.23 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics= ['accuracy'])

In [39]:
model.fit(training_image_data, 
          validation_data= testing_image_data,
          epochs = 200,
          validation_steps= (len(testing_image_data.filenames) // testing_image_data.batch_size),
          steps_per_epoch = (len(training_image_data.filenames) // training_image_data.batch_size),
          callbacks = MyCLRuleMonitor(0.7))

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.4000 - loss: 0.7005 - val_accuracy: 0.4500 - val_loss: 0.7080
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6364 - loss: 0.6829 - val_accuracy: 0.5000 - val_loss: 0.7161
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.4545 - loss: 0.7305 - val_accuracy: 0.4500 - val_loss: 0.6893
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8500 - loss: 0.6677 - val_accuracy: 0.5500 - val_loss: 0.6812
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.6500 - loss: 0.6508 - val_accuracy: 0.5500 - val_loss: 0.6818
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5455 - loss: 0.6419 - val_accuracy: 0.5500 - val_loss: 0.6868
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5455 - loss: 0.6821 - val_accuracy: 0.4500 - val_loss: 0.7038
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8500 - loss: 0.5786 - val_accuracy: 0.4500 - val_loss: 0.7031
Epoch 

In [ ]:
# Testing | Deployment
# Read the image - convert to np array - add dimention using np - get prediction - change prediction aproximation to value

image = tf.keras.preprocessing.image.load_img('celeb/data/validation/salman/salman-khan_1.jpg', target_size = (128,128))

image_arr = tf.keras.preprocessing.image.img_to_array(image)

np_image_arr = np.expand_dims(image_arr, axis = 0)

predictions = model.predict(np_image_arr)
print(predictions)

# 2. Find the index of the class with the highest probability
predicted_class_index = np.argmax(predictions) #

# print(predicted_class_index)
# print(training_image_data.classes)
# 3. Create a reverse mapping dictionary to get the class name from the index
# Original: {'class_name': index} -> Reversed: {index: 'class_name'}
class_names_map = {v: k for k, v in training_image_data.class_indices.items()}

# print(class_names_map)
# 4. Use the index to look up the actual class name
predicted_class_name = class_names_map[predicted_class_index]

# 5. Print the result
# print(f"The predicted celeb name is: {predicted_class_name}") #

# {0: 'salman', 1: 'shahrukh'}

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[[1.]]
